In [4]:
from __future__ import division
from __future__ import print_function
import pandas as pd
import bokeh
import numpy as np
import sys
import os

from datetime import datetime
from datetime import timedelta

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, os.pardir)
sys.path.append(src_dir)

import read_proximity

In [5]:
# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
# list of modules to autoload
%aimport read_proximity

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# check gaps between scans - by resampling to minute interval

In [16]:
# test data
#test_data = read_proximity.proximity2data("../../../../Downloads/prox.txt")


# load file 1
pi10_data = read_proximity.proximity2data("../../../../Downloads/pi10_log_proximity.txt")

# load file 2
pi11_data = read_proximity.proximity2data("../../../../Downloads/pi11_log_proximity.txt")

# resample, by minute - that's how often it shoudl have be done
# look for nulls?


12
678
607


In [52]:
test_data = pd.concat([pi10_data,pi11_data])
print(test_data.head())
print(test_data.tail())

                         badge_address      member
datetime                                          
2016-10-12 14:15:41  C1:27:66:94:0F:0A  1KO4K0032H
2016-10-12 14:16:41  C1:27:66:94:0F:0A  1KO4K0032H
2016-10-12 14:17:41  C1:27:66:94:0F:0A  1KO4K0032H
2016-10-12 14:17:36  D6:B8:C6:BA:9B:2B  CLZFQZJYW4
2016-10-12 14:18:36  D6:B8:C6:BA:9B:2B  CLZFQZJYW4
                         badge_address      member
datetime                                          
2016-10-12 22:47:44  C1:27:66:94:0F:0A  1KO4K0032H
2016-10-12 22:48:19  D6:B8:C6:BA:9B:2B  CLZFQZJYW4
2016-10-12 22:49:19  D6:B8:C6:BA:9B:2B  CLZFQZJYW4
2016-10-12 22:49:44  C1:27:66:94:0F:0A  1KO4K0032H
2016-10-12 22:50:19  D6:B8:C6:BA:9B:2B  CLZFQZJYW4


In [53]:
del test_data['member']
test_data['exist'] = 1

In [54]:
grouped = test_data.groupby('badge_address')
print(grouped.head())

                         badge_address  exist
datetime                                     
2016-10-12 14:15:41  C1:27:66:94:0F:0A      1
2016-10-12 14:16:41  C1:27:66:94:0F:0A      1
2016-10-12 14:17:41  C1:27:66:94:0F:0A      1
2016-10-12 14:17:36  D6:B8:C6:BA:9B:2B      1
2016-10-12 14:18:36  D6:B8:C6:BA:9B:2B      1
2016-10-12 14:18:41  C1:27:66:94:0F:0A      1
2016-10-12 14:19:36  D6:B8:C6:BA:9B:2B      1
2016-10-12 14:22:37  D6:B8:C6:BA:9B:2B      1
2016-10-12 14:22:41  C1:27:66:94:0F:0A      1
2016-10-12 14:23:37  D6:B8:C6:BA:9B:2B      1


In [55]:
df_resampled = grouped.resample(rule='60S').sum()
print(df_resampled)

                                       exist
badge_address     datetime                  
C1:27:66:94:0F:0A 2016-10-12 14:15:00    1.0
                  2016-10-12 14:16:00    2.0
                  2016-10-12 14:17:00    1.0
                  2016-10-12 14:18:00    1.0
                  2016-10-12 14:19:00    1.0
                  2016-10-12 14:20:00    1.0
                  2016-10-12 14:21:00    1.0
                  2016-10-12 14:22:00    1.0
                  2016-10-12 14:23:00    1.0
                  2016-10-12 14:24:00    1.0
                  2016-10-12 14:25:00    1.0
                  2016-10-12 14:26:00    1.0
                  2016-10-12 14:27:00    2.0
                  2016-10-12 14:28:00    1.0
                  2016-10-12 14:29:00    1.0
                  2016-10-12 14:30:00    1.0
                  2016-10-12 14:31:00    1.0
                  2016-10-12 14:32:00    1.0
                  2016-10-12 14:33:00    1.0
                  2016-10-12 14:34:00    1.0
          

In [56]:
print(len(df_resampled))
print(len(df_resampled[df_resampled['exist']>1.0]))
print(len(df_resampled[df_resampled['exist'].notnull()]))

1205
81
1204


In [57]:
df_resampled[df_resampled['exist'].isnull()]

,,exist
badge_address,datetime,
D6:B8:C6:BA:9B:2B,2016-10-12 19:50:00,NaN


In [58]:
# so, there is only one missing minute. However, this might be a result of innacuracies in when scans are done/saved
# a better approach might be to check the time between samples?

# check time between samples

In [60]:
# group by badge
# remove duplicates
# look at code from other tests for how I check for time between samples?